In [ ]:
import boto3
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import ipywidgets as widgets
from IPython.display import display

## S3 Bucket Name into Global variable from interactive text box

In [ ]:
# Create an interactive text widget for entering the S3 bucket name

import ipywidgets as widgets
from IPython.display import display

bucket_name_widget = widgets.Text(
    value='',
    placeholder='Enter S3 bucket',
    description='Bucket:',
    disabled=False
)
button = widgets.Button(description="Set Bucket Name")
output = widgets.Output()

def on_button_click(b):
    global bucket_name
    bucket_name = bucket_name_widget.value
    with output:
        output.clear_output()
        print(f"S3 bucket name set to: {bucket_name}")

button.on_click(on_button_click)
ui = widgets.HBox([bucket_name_widget, button])
display(ui, output)


## Display first 5 DICOM images

In [ ]:
import boto3
import io

def fetch_dicom_files(bucket_name):
    s3 = boto3.client('s3')
    try:
        response = s3.list_objects_v2(Bucket=bucket_name)
        if 'Contents' not in response:
            return [], 'No DICOM files found in the bucket.'

        dicom_files = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.dcm')]
        return dicom_files, None
    except Exception as e:
        return [], str(e)

dicom_files, error = fetch_dicom_files(bucket_name)

if error:
    print(f"Error fetching DICOM files: {error}")
else:
    if len(dicom_files) == 0:
        print("No DICOM files found in the bucket.")
    else:
        print(f"Successfully fetched {len(dicom_files)} DICOM files.")
        print("Here are the first few DICOM files:")
        for dicom_file in dicom_files[:5]:  
            print(dicom_file)


## Visualize DICOM image via dropdown list selection

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output

if dicom_files:
    dropdown = widgets.Dropdown(
        options=dicom_files,
        description='Select DICOM:',
        disabled=False,
    )
    
    def on_select_change(change):
        clear_output(wait=True)
        display(dropdown)
        visualize_dicom(bucket_name, change['new'])
    
    dropdown.observe(on_select_change, names='value')
    display(dropdown)
else:
    print("No DICOM files available for selection.")


## Create sliders for contrast and brightness

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import io
import boto3
from IPython.display import clear_output, display

# Sliders for contrast and brightness
contrast_slider = widgets.FloatSlider(
    value=1.0,
    min=0.5,
    max=2.0,
    step=0.1,
    description='Contrast:',
    continuous_update=False
)

brightness_slider = widgets.FloatSlider(
    value=0.0,
    min=-100,
    max=100,
    step=10,
    description='Brightness:',
    continuous_update=False
)

def download_dicom_file(bucket_name, dicom_key):
    s3 = boto3.client('s3')
    try:
        dicom_obj = s3.get_object(Bucket=bucket_name, Key=dicom_key)
        dicom_data = dicom_obj['Body'].read()
        dicom_file = pydicom.dcmread(io.BytesIO(dicom_data))
        return dicom_file, None
    except Exception as e:
        return None, str(e)

def update_image(change):
    clear_output(wait=True)  # Clear previous images and sliders
    display(contrast_slider, brightness_slider)  # Redisplay sliders
    
    dicom_data, error = download_dicom_file(bucket_name, dropdown.value)
    if dicom_data:
        image = dicom_data.pixel_array
        # Apply contrast and brightness adjustments
        adjusted_image = np.clip(contrast_slider.value * image + brightness_slider.value, 0, 255)

        # Display the adjusted image
        plt.figure(figsize=(6, 6))
        plt.imshow(adjusted_image, cmap=plt.cm.gray)
        plt.title(f"Selected DICOM File: {dropdown.value}", fontsize=12)
        plt.axis('off')
        plt.show()
    else:
        print(f"Error: {error}")

contrast_slider.observe(update_image, names='value')
brightness_slider.observe(update_image, names='value')
display(contrast_slider, brightness_slider)
update_image(None)


## Histogram of Pixel Intensity

In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt

# Re-importing ipywidgets to ensure functionality (optional)
import ipywidgets as widgets
from IPython.display import display

histogram_button = widgets.Button(description="Show Histogram")
histogram_output = widgets.Output()

def show_histogram(b):
    with histogram_output:
        histogram_output.clear_output()
    
        dicom_data, error = download_dicom_file(bucket_name, dropdown.value)
        if dicom_data:
            # Get the pixel array from the DICOM file
            pixel_array = dicom_data.pixel_array

            plt.figure(figsize=(6, 4))
            plt.hist(pixel_array.ravel(), bins=256, color='gray', alpha=0.75)
            plt.title(f'Pixel Intensity Histogram\nSelected DICOM File: {dropdown.value}', fontsize=12)
            plt.xlabel('Pixel Intensity')
            plt.ylabel('Frequency')
            plt.show()
        else:
            print(f"Error downloading DICOM file: {error}")


histogram_button.on_click(show_histogram)
display(histogram_button, histogram_output)


## Overlay with dropdown selection for colormap

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import colors
import ipywidgets as widgets
from IPython.display import display

custom_cmap = colors.LinearSegmentedColormap.from_list(
    'custom_cool', ['#0000FF', '#FF0000']  # From blue to red
)

dropdown1 = widgets.Dropdown(
    options=dicom_files,
    value=dropdown.value,  # Set to the last selected image
    description='DICOM 1:',
    disabled=False,
)

dropdown2 = widgets.Dropdown(
    options=dicom_files,
    description='DICOM 2:',
    disabled=False,
)

colormap_dropdown = widgets.Dropdown(
    options={'Gray': 'gray', 'Cool': 'cool', 'Hot': 'hot', 'Custom Cool': custom_cmap},
    value='gray',  # Default to 'gray'
    description='Colormap:',
)

overlay_button = widgets.Button(description="Overlay Images")
overlay_output = widgets.Output()

def overlay_images(b):
    dicom_data1, error1 = download_dicom_file(bucket_name, dropdown1.value)
    dicom_data2, error2 = download_dicom_file(bucket_name, dropdown2.value)
    
    if error1 or error2:
        print(f"Error downloading DICOM files: {error1 or error2}")
    else:
        image1 = dicom_data1.pixel_array
        image2 = dicom_data2.pixel_array

        image1 = image1 / np.max(image1)
        image2 = image2 / np.max(image2)
        
        # Overlay the images (simple averaging)
        overlay = (image1 + image2) / 2
        
        with overlay_output:
            overlay_output.clear_output()
            plt.figure(figsize=(6, 6))
            plt.imshow(overlay, cmap=colormap_dropdown.value)
            plt.title(f"Overlay of {dropdown1.value} and {dropdown2.value}", fontsize=12)
            plt.axis('off')
            plt.show()

overlay_button.on_click(overlay_images)
display(dropdown1, dropdown2, colormap_dropdown, overlay_button, overlay_output)
